# import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time
import os

%matplotlib inline
from utils import config
from utils.backtest import backtest_stats, backtest_plot, get_baseline
from utils.preprocessors import split_data
from utils.preprocessors import config

c:\users\w\appdata\local\programs\python\python39\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


min_variance默认采用回溯一年的时间进行协方差分析
max_sharpe回溯一年效果不好

# 等权重投资策略——equal-weighted strategy

In [2]:
trade=pd.read_csv('./learn/data_file/trade.csv')
trade.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,close_20_sma,close_60_sma,close_120_sma,macd,volume_20_sma,volume_60_sma,volume_120_sma,amount,change,daily_variance
0,2020/1/2,600000.SH,11.2577,11.4112,11.2397,11.2577,516290.79,3,11.387935,10.624635,...,11.006285,11.167883,10.809557,0.060492,3.164282e+05,4.204398e+05,3.674288e+05,5.812247e+06,0.000000,0.015234
1,2020/1/2,600009.SH,77.9850,78.1137,76.3220,76.5793,158265.63,3,78.618465,76.031035,...,77.324750,76.529690,78.919330,0.197284,7.282439e+04,8.589628e+04,8.302769e+04,1.211987e+07,-0.018356,0.023397
2,2020/1/2,600016.SH,5.7133,5.7492,5.7043,5.7133,1043664.80,3,5.713458,5.527192,...,5.620325,5.566270,5.459445,0.032701,5.945865e+05,5.888251e+05,5.792997e+05,5.962770e+06,0.000000,0.007859
3,2020/1/2,600028.SH,4.5208,4.5737,4.5032,4.5561,1250435.92,3,4.538989,4.305251,...,4.422120,4.387307,4.403891,0.033443,5.657396e+05,5.319938e+05,5.919345e+05,5.697111e+06,0.007748,0.015474
4,2020/1/2,600030.SH,24.9261,25.0714,24.4613,24.6840,3577365.90,3,24.731324,20.567536,...,22.649430,21.781283,21.872575,0.703307,1.754846e+06,1.038214e+06,1.061408e+06,8.830370e+07,-0.009808,0.024716


In [4]:
trade['date'] = pd.to_datetime(trade['date'])

In [5]:
trade.shape

(24300, 21)

In [6]:
unique_tic=trade.tic.unique()
unique_trade_date=trade.date.unique()

In [7]:
len(unique_tic)

50

In [8]:
len(unique_trade_date)

486

In [9]:
initial_capital=1000000
every_tic_capital=initial_capital/len(unique_tic)
holding_list=[]
initial_close=trade[trade['date']==unique_trade_date[0]]['close'].to_list()
for i in range(len(unique_tic)):
    if initial_close[i]==0:
        holding_list.append(0)
    else:
        holding_list.append(every_tic_capital/initial_close[i])
total_assets_list=[initial_capital]
holdings=np.array(holding_list)
for i in range(1,len(unique_trade_date)):
    close=trade[trade['date']==unique_trade_date[i]]['close'].values
    assets=np.dot(holdings,close)
    total_assets_list.append(assets)

In [10]:
total_assets_list

[1000000,
 958480.250888619,
 956245.2025972549,
 958113.3963361068,
 949007.5131767603,
 961255.5236598668,
 965130.5301986268,
 975806.874915252,
 970055.6366753449,
 968284.6566280174,
 963896.383312491,
 968154.3041751551,
 979383.254368342,
 966477.1793089488,
 975595.5558089952,
 945508.0999701293,
 870458.8525204696,
 899003.6071808165,
 906834.4135357143,
 925577.294355114,
 932425.3594691057,
 932798.1215344731,
 939386.6405673993,
 951395.0501088263,
 951852.8269515864,
 954519.0525707299,
 977816.3067682165,
 978865.6081420172,
 977682.1450318828,
 1002444.3306652876,
 1006780.3594964544,
 1009423.7793417163,
 1007716.5564576364,
 985462.6308243354,
 987661.3650641339,
 947688.9703142821,
 975846.2910869283,
 983812.7379173638,
 981160.1608585842,
 1000510.4373303765,
 982753.4689825978,
 940156.605396707,
 963736.4696349556,
 946133.163417487,
 906550.7555984503,
 898320.361009797,
 856865.5308708363,
 855419.2195467011,
 838941.9370405349,
 829117.9452659058,
 843080.82351

In [40]:
equal_weighted=pd.DataFrame({'total_assets':total_assets_list})
equal_weighted.to_csv('./learn/trade_file/equal_weight.csv',index=0)

# Data preprocess

In [2]:
from pypfopt import expected_returns
df=pd.read_csv('./learn/data_file/data.csv')

In [3]:
df.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,close_20_sma,close_60_sma,close_120_sma,macd,volume_20_sma,volume_60_sma,volume_120_sma,amount,change,daily_variance
0,2015/1/5,600000.SH,8.8185,9.0239,8.6408,8.9240,5135687.09,0,0.0,0.0,...,8.9240,8.9240,8.9240,0.0,5135687.09,5135687.09,5135687.09,4.583087e+07,0.011822,0.042929
1,2015/1/5,600009.SH,18.5425,19.5623,18.5425,19.2068,371485.54,0,0.0,0.0,...,19.2068,19.2068,19.2068,0.0,371485.54,371485.54,371485.54,7.135048e+06,0.034587,0.053096
2,2015/1/5,600016.SH,6.9437,7.0012,6.7073,6.8862,9138873.70,0,0.0,0.0,...,6.8862,6.8862,6.8862,0.0,9138873.70,9138873.70,9138873.70,6.293211e+07,-0.008350,0.042680
3,2015/1/5,600028.SH,4.4782,4.8520,4.3831,4.8520,11864996.45,0,0.0,0.0,...,4.8520,4.8520,4.8520,0.0,11864996.45,11864996.45,11864996.45,5.756896e+07,0.077040,0.096641
4,2015/1/5,600030.SH,29.6243,30.8041,28.8466,30.2885,6986272.15,0,0.0,0.0,...,30.2885,30.2885,30.2885,0.0,6986272.15,6986272.15,6986272.15,2.116037e+08,0.021929,0.064628


In [4]:
df.date

0          2015/1/5
1          2015/1/5
2          2015/1/5
3          2015/1/5
4          2015/1/5
            ...    
85245    2021/12/31
85246    2021/12/31
85247    2021/12/31
85248    2021/12/31
85249    2021/12/31
Name: date, Length: 85250, dtype: object

In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]
df.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,close_20_sma,close_60_sma,close_120_sma,macd,volume_20_sma,volume_60_sma,volume_120_sma,amount,change,daily_variance
0,2015-01-05,600000.SH,8.8185,9.0239,8.6408,8.9240,5135687.09,0,0.0,0.0,...,8.9240,8.9240,8.9240,0.0,5135687.09,5135687.09,5135687.09,4.583087e+07,0.011822,0.042929
0,2015-01-05,600009.SH,18.5425,19.5623,18.5425,19.2068,371485.54,0,0.0,0.0,...,19.2068,19.2068,19.2068,0.0,371485.54,371485.54,371485.54,7.135048e+06,0.034587,0.053096
0,2015-01-05,600016.SH,6.9437,7.0012,6.7073,6.8862,9138873.70,0,0.0,0.0,...,6.8862,6.8862,6.8862,0.0,9138873.70,9138873.70,9138873.70,6.293211e+07,-0.008350,0.042680
0,2015-01-05,600028.SH,4.4782,4.8520,4.3831,4.8520,11864996.45,0,0.0,0.0,...,4.8520,4.8520,4.8520,0.0,11864996.45,11864996.45,11864996.45,5.756896e+07,0.077040,0.096641
0,2015-01-05,600030.SH,29.6243,30.8041,28.8466,30.2885,6986272.15,0,0.0,0.0,...,30.2885,30.2885,30.2885,0.0,6986272.15,6986272.15,6986272.15,2.116037e+08,0.021929,0.064628


In [7]:
def replace_zero(x):
    if x==0:
        return np.nan
    else:
        return x
df['close']=df['close'].apply(replace_zero)
df=df.fillna(method='ffill')  #沿着行填充
df.shape

(85250, 21)

In [8]:
# add return and covariance matrix as states
cov_list = []
return_list = []
mu_list=[]

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  mu=expected_returns.mean_historical_return(price_lookback)
  mu_list.append(mu)
  return_lookback = price_lookback.pct_change().dropna(how='all').fillna(axis=1,method='ffill')
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list,'mu_list':mu_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [9]:
df.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,macd,volume_20_sma,volume_60_sma,volume_120_sma,amount,change,daily_variance,cov_list,return_list,mu_list
0,2016-01-14,600000.SH,9.8811,10.4565,9.8171,10.4158,548388.33,3,11.407182,9.838818,...,-0.162924,5.496728e+05,8.691621e+05,1.289144e+06,5.711903e+06,0.051335,0.061388,"[[0.01268335102170229, 0.0008668331707704101, ...",tic 600000.SH 600009.SH 600016.SH 6...,tic 600000.SH 0.167167 600009.SH 0.32429...
1,2016-01-14,600009.SH,25.1326,25.6059,24.6120,25.4355,122526.75,3,30.397581,24.810859,...,-0.777367,9.787863e+04,1.045091e+05,1.624864e+05,3.116529e+06,0.011909,0.039075,"[[0.01268335102170229, 0.0008668331707704101, ...",tic 600000.SH 600009.SH 600016.SH 6...,tic 600000.SH 0.167167 600009.SH 0.32429...
2,2016-01-14,600016.SH,5.3924,5.5485,5.3664,5.5225,649121.67,3,6.497090,5.315440,...,-0.078695,2.015290e+06,1.481336e+06,1.662394e+06,3.584774e+06,0.023558,0.032974,"[[0.01268335102170229, 0.0008668331707704101, ...",tic 600000.SH 600009.SH 600016.SH 6...,tic 600000.SH 0.167167 600009.SH 0.32429...
3,2016-01-14,600028.SH,3.2305,3.2726,3.1813,3.2585,1460281.16,3,3.651230,3.196640,...,-0.068620,1.493799e+06,1.737994e+06,2.271742e+06,4.758326e+06,0.008593,0.028019,"[[0.01268335102170229, 0.0008668331707704101, ...",tic 600000.SH 600009.SH 600016.SH 6...,tic 600000.SH 0.167167 600009.SH 0.32429...
4,2016-01-14,600030.SH,13.7175,14.4875,13.5228,14.4167,2060976.50,3,19.487388,13.117882,...,-0.688657,2.596742e+06,3.034415e+06,2.821732e+06,2.971248e+07,0.048499,0.066915,"[[0.01268335102170229, 0.0008668331707704101, ...",tic 600000.SH 600009.SH 600016.SH 6...,tic 600000.SH 0.167167 600009.SH 0.32429...


In [10]:
df.shape

(72650, 24)

In [11]:
trade=split_data(df,config.End_Trade_Date,config.End_Test_Date)
trade.shape

(24300, 24)

In [12]:
list(trade.date.unique())

[numpy.datetime64('2020-01-02T00:00:00.000000000'),
 numpy.datetime64('2020-01-03T00:00:00.000000000'),
 numpy.datetime64('2020-01-06T00:00:00.000000000'),
 numpy.datetime64('2020-01-07T00:00:00.000000000'),
 numpy.datetime64('2020-01-08T00:00:00.000000000'),
 numpy.datetime64('2020-01-09T00:00:00.000000000'),
 numpy.datetime64('2020-01-10T00:00:00.000000000'),
 numpy.datetime64('2020-01-13T00:00:00.000000000'),
 numpy.datetime64('2020-01-14T00:00:00.000000000'),
 numpy.datetime64('2020-01-15T00:00:00.000000000'),
 numpy.datetime64('2020-01-16T00:00:00.000000000'),
 numpy.datetime64('2020-01-17T00:00:00.000000000'),
 numpy.datetime64('2020-01-20T00:00:00.000000000'),
 numpy.datetime64('2020-01-21T00:00:00.000000000'),
 numpy.datetime64('2020-01-22T00:00:00.000000000'),
 numpy.datetime64('2020-01-23T00:00:00.000000000'),
 numpy.datetime64('2020-02-03T00:00:00.000000000'),
 numpy.datetime64('2020-02-04T00:00:00.000000000'),
 numpy.datetime64('2020-02-05T00:00:00.000000000'),
 numpy.datet

# Min-Variance strategy

In [22]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [23]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [24]:
len(unique_tic)

50

In [25]:
len(unique_trade_date)

486

In [27]:
#计算最小方差的投资组合
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital
last_cleaned_weights_min_var={}
for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    try:
    #portfolio allocation
        ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
        #minimum variance
        raw_weights_min_var = ef_min_var.min_volatility()
        #get weights
        cleaned_weights_min_var = ef_min_var.clean_weights()
        last_cleaned_weights_min_var=cleaned_weights_min_var
    except:
        print(f'日期为{unique_trade_date[i]},索引为{i}')
        print(last_cleaned_weights_min_var)
        cleaned_weights_min_var=last_cleaned_weights_min_var
    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value 
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)
    
portfolio=portfolio.T
portfolio.columns = ['total_assets']

日期为2020-07-09T00:00:00.000000000,索引为123
OrderedDict([('600000.SH', 0.03412), ('600009.SH', 0.02154), ('600016.SH', 0.1), ('600028.SH', 0.1), ('600030.SH', 0.0), ('600031.SH', 0.0), ('600036.SH', 0.0), ('600048.SH', 0.0), ('600050.SH', 0.0), ('600104.SH', 0.0), ('600196.SH', 0.07565), ('600276.SH', 0.07646), ('600309.SH', 0.0), ('600438.SH', 0.0), ('600519.SH', 0.0), ('600547.SH', 0.07462), ('600570.SH', 0.0), ('600585.SH', 0.00305), ('600588.SH', 0.0), ('600690.SH', 0.0), ('600703.SH', 0.0), ('600745.SH', 0.0), ('600809.SH', 0.0), ('600837.SH', 0.0), ('600887.SH', 0.00343), ('600893.SH', 0.01731), ('600918.SH', 0.0), ('601012.SH', 0.0), ('601066.SH', 0.0), ('601088.SH', 0.05716), ('601138.SH', 0.0), ('601166.SH', 0.0), ('601211.SH', 0.0), ('601288.SH', 0.1), ('601318.SH', 0.0), ('601336.SH', 0.0), ('601398.SH', 0.1), ('601601.SH', 0.0), ('601628.SH', 0.0), ('601668.SH', 0.0), ('601688.SH', 0.0), ('601818.SH', 0.03642), ('601857.SH', 0.1), ('601888.SH', 0.0), ('601899.SH', 0.00012), ('6

In [28]:
portfolio.head()

,total_assets
2020-01-02,1000000
2020-01-03,1009377.270369
2020-01-06,1020710.795932
2020-01-07,1019157.870281
2020-01-08,1020128.9368


In [29]:
portfolio.to_csv('./learn/trade_file/min_variance.csv',index=0)

## max-sharpe strategy

In [35]:
cost_pct=3e-3  #无论买卖应该支付的交易费用

In [36]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [37]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [18]:
#计算最大夏普比率的投资组合
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital
last_cleaned_weights_max_sharpe={}
last_ideal_cash=[0 for i in range(50)]
for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    expect_return=df_temp.mu_list[0]
    try:
    #portfolio allocation
        ef = EfficientFrontier(expect_return, Sigma,weight_bounds=(0, 0.1))
        #minimum variance
        raw_weights_max_sharpe = ef.max_sharpe()
        #get weights
        cleaned_weights_max_sharpe = ef.clean_weights()
        last_cleaned_weights_max_sharpe=cleaned_weights_max_sharpe
    except:
        print(f'日期为{unique_trade_date[i]},索引为{i}')
        print(last_cleaned_weights_max_sharpe)
        cleaned_weights_max_sharpe=last_cleaned_weights_max_sharpe
    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_max_sharpe.values())]
    #引入交易费用
    cash_diff=np.abs(np.array(current_cash)-np.array(last_ideal_cash))
    buy_sell_cost=cash_diff*cost_pct
    last_ideal_cash=list(np.array(current_cash)-buy_sell_cost)
    #print(cash_diff)
    # actual current held shares 
    current_shares = list((np.array(current_cash)-buy_sell_cost)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value 
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)
    
portfolio=portfolio.T
portfolio.columns = ['total_assets']

In [19]:
portfolio.head()

,total_assets
2020-01-02,1000000
2020-01-03,988946.300847
2020-01-06,987850.919216
2020-01-07,982446.278225
2020-01-08,979106.209248


In [20]:
portfolio.to_csv('./learn/trade_file/max_sharpe.csv',index=0)

## 最大化二次效用函数

In [21]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
cost_pct=3e-3  #无论买卖应该支付的交易费用
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [26]:
#计算最大效用函数下的各资产权重
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital
last_cleaned_weights_max_sharpe={}
last_ideal_cash=[0 for i in range(50)]
for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    expect_return=df_temp.mu_list[0]
    try:
    #portfolio allocation
        ef = EfficientFrontier(expect_return, Sigma,weight_bounds=(0, 0.1))
        #minimum variance
        raw_weights_max_sharpe = ef.max_quadratic_utility(risk_aversion=0.5)
        #get weights
        cleaned_weights_max_sharpe = ef.clean_weights()
        last_cleaned_weights_max_sharpe=cleaned_weights_max_sharpe
    except:
        print(f'日期为{unique_trade_date[i]},索引为{i}')
        print(last_cleaned_weights_max_sharpe)
        cleaned_weights_max_sharpe=last_cleaned_weights_max_sharpe
    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_max_sharpe.values())]
    #引入交易费用
    cash_diff=np.abs(np.array(current_cash)-np.array(last_ideal_cash))
    buy_sell_cost=cash_diff*cost_pct
    last_ideal_cash=list(np.array(current_cash)-buy_sell_cost)
    #print(cash_diff)
    # actual current held shares 
    current_shares = list((np.array(current_cash)-buy_sell_cost)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value 
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)
    
portfolio=portfolio.T
portfolio.columns = ['total_assets']

In [27]:
portfolio.head()

,total_assets
2020-01-02,1000000
2020-01-03,982540.837513
2020-01-06,973314.034657
2020-01-07,971968.980177
2020-01-08,964220.419456


In [28]:
portfolio.to_csv('./learn/trade_file/max_utility.csv',index=0)